# RAG pipeline

In [58]:
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import FAISS
import pandas as pd
import psycopg2
from langchain_huggingface import HuggingFaceEmbeddings


In [59]:
# Step 4: Initialize LLaMA LLM via Ollama
# freeze
llm = Ollama(model="llama3.1:latest")
# llm = Ollama(model="llama3.1:70b")


In [60]:
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

bge_m3_ef = BGEM3EmbeddingFunction(
    model_name='BAAI/bge-m3', # Specify the model name
    device='cpu', # Specify the device to use, e.g., 'cpu' or 'cuda:0'
    use_fp16=False # Specify whether to use fp16. Set to `False` if `device` is `cpu`.
)

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 18930.21it/s]
/Volumes/home/Learn/Master/3.LLMs/rag_pipeline/.venv/lib/python3.11/site-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on

## Milvus standalone

In [61]:
from pymilvus import MilvusClient

client = MilvusClient(
    uri="http://localhost:19530"
)

In [62]:
# if client.has_collection(collection_name="demo_collection"):
#     client.drop_collection(collection_name="demo_collection")
# client.create_collection(
#         collection_name="demo_collection",
#         dimension=1024,  # The vectors we will use in this demo has 768 dimensions
# )


In [63]:
import os
import pandas as pd

# Đường dẫn đến thư mục "Khoi"
folder_path = '../Khoi'

# Danh sách lưu trữ các DataFrame
df_list = []

# Duyệt qua tất cả các file trong thư mục
# for file_name in os.listdir('Khoi'):
for file_name in ['Văn Bản Pháp Luật_p100000-100019.parquet']:
    # Kiểm tra nếu file có định dạng giống "Văn Bản Pháp Luật_p*.parquet"
    print("file_name", file_name)
    file_path = os.path.join(folder_path, file_name)
    
    # Đọc file Parquet và thêm vào danh sách DataFrame
    file_df = pd.read_parquet(file_path)
    
    # Thực hiện các chuyển đổi tương tự như bạn đã đề cập
    file_df['created_date'] = pd.to_datetime(file_df['created_date'], format='%d/%m/%Y')
    file_df['updated_date'] = pd.to_datetime(file_df['updated_date'], format='%d/%m/%Y')
    file_df['content_raw'] = file_df['content'].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
    
    # df_list.append(file_df)
    
    

    # connection = create_connection()

    # cursor = connection.cursor()

    try: 
        # Chèn dữ liệu từ DataFrame vào PostgreSQL
        
        data = []
        for index, item in file_df.iterrows():
            print(item)
            docs= [f"{item.title}"]
            
            docs_embeddings = bge_m3_ef.encode_documents(docs)

            vectors = docs_embeddings["dense"]

        
            data.append({"id": item.id, "vector": vectors[0], "text": docs[0],"url":item.url,"title": item.title, "subject": "van_ban_phap_luat"} )
        
        res = client.insert(collection_name="demo_collection", data=data)
        print(res)
        
        # Xác nhận các thay đổi vào cơ sở dữ liệu
        # connection.commit()

    except Exception as e:
        print(e)

    # Đóng kết nối
    # cursor.close()
    # connection.close()

    print("Data inserted successfully!")


# # Nối tất cả các DataFrame lại với nhau thành một DataFrame duy nhất
# combined_df = pd.concat(df_list, ignore_index=True)

# # Hiển thị vài dòng đầu tiên của DataFrame
# combined_df.head()

file_name Văn Bản Pháp Luật_p100000-100019.parquet
id                                                         100000
url             https://thuvienphapluat.vn/van-ban/Quyen-dan-s...
title           Quyết định 720/QĐ-CTN năm 2020 về cho thôi quố...
created_date                                  2020-05-18 00:00:00
updated_date                                  2020-06-03 00:00:00
content         b'<div class="content1">\n <div>\n  <div>\n   ...
content_raw     <div class="content1">\n <div>\n  <div>\n   <t...
Name: 0, dtype: object
id                                                         100001
url             https://thuvienphapluat.vn/van-ban/Quyen-dan-s...
title           Quyết định 719/QĐ-CTN năm 2020 về cho thôi quố...
created_date                                  2020-05-18 00:00:00
updated_date                                  2020-06-03 00:00:00
content         b'<div class="content1">\n <div>\n  <div>\n   ...
content_raw     <div class="content1">\n <div>\n  <div>\n   <t...
Na

In [64]:

# res = client.query(
#     collection_name="demo_collection",  # target collection
#     filter='',  # number of returned entities
#     limit=100,
#     output_fields=["id", 'text'],  # specifies fields to be returned
#     # sorted=True, #
# )
# for item in res:
#     print(item)

In [65]:
# queries = ["năm 2020 về cho thôi quốc tịch Việt Nam"]

# query_embeddings = bge_m3_ef.encode_queries(queries)

# res = client.search(
#     collection_name="demo_collection",  # target collection
#     data=[query_embeddings['dense'][0]],  # query vectors
#     limit=5,  # number of returned entities
#     output_fields=["text", "subject"],  # specifies fields to be returned
# )
# context_items = res[0]

# print(context_items)



In [66]:
context = [item['entity']['text'] for item in context_items]
context 

['Quyết định 720/QĐ-CTN năm 2020 về cho thôi quốc tịch Việt Nam đối với 95 công dân hiện đang cư trú tại Đức do Chủ tịch nước ban hành']

In [67]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [68]:
prompt = '''
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question or history of the chat. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
History:{history}
Answer: 

'''

In [69]:
history = []

In [70]:
example_messages = llm.invoke(prompt)

In [71]:
example_messages

'There is no context provided. What would you like me to do?'

In [72]:
def question_llm(question):
  prompt_formatted = prompt.format(question=question, context=context, history=history)
  res = llm.invoke(prompt_formatted)
  history.append([
    ('user',question),
    ('system',res)
  ])
  
  # print(prompt_formatted)
  print(res)

In [73]:
question_llm("Quyết định 720/QĐ-CTN năm 2020 là gì")

Quyết định 720/QĐ-CTN năm 2020 là quyết định của Chủ tịch nước Việt Nam về việc cho thôi quốc tịch Việt Nam đối với 95 công dân. Họ hiện đang cư trú tại Đức. Quyết định này được ban hành vào năm 2020.


In [74]:
question_llm("Bạn lấy thông tin từ đâu")

Tôi lấy thông tin từ các văn bản pháp luật, trong trường hợp này là Quyết định 720/QĐ-CTN năm 2020. Cụ thể hơn, thông tin này được thu thập từ trang web của Chủ tịch nước Việt Nam hoặc các cơ quan liên quan đến vấn đề quốc tịch.


In [75]:
question_llm("Câu hỏi về Quyết định 720/QĐ-CTN khi nãy bạn lấy thông tin từ dữ liệu nào")

Tôi lấy thông tin từ văn bản pháp luật, cụ thể là Quyết định 720/QĐ-CTN năm 2020 và trang web của Chủ tịch nước Việt Nam. Thông tin này được thu thập từ các cơ quan liên quan đến vấn đề quốc tịch.


In [76]:
question_llm("Thể hiện các context mà bạn thao khảo")

Tôi không có thông tin về câu hỏi tiếp theo của bạn trong cuộc trò chuyện này.


## FAQ
Khi nhận câu hỏi -> Hệ thống detect FAQ -> Trả lời FAQ answer

==> Người dùng sửa câu trả lời --> Lưu 

==> Nếu có version của user khác sửa rồi thì cho user đánh giá cái nào tốt hơn (random 2 cái)

Sau 1 khoảng tg câu trả lời nào điểm cao thì update vào FAQ

In [77]:
# client.drop_collection(collection_name="faq_collection")



In [78]:

# client.create_collection(
#         collection_name="faq_collection",
#         dimension=1024,  # The vectors we will use in this demo has 768 dimensions
#         auto_id=True
# )

In [79]:
# data = [
#   {'question': 'Quyết định 720/QĐ-CTN năm 2020 là gì', 'answer': '1'},
#   {'question': 'Quyết định 720/QĐ-CTN năm 2020 có gì', 'answer': '2'},
#   {'question': 'Tôi yêu em', 'answer': '3'},
#   ]
# docs= [f"{item['question']}" for item in data ]
            
# docs_embeddings = bge_m3_ef.encode_documents(docs)

# vectors = docs_embeddings["dense"]

# data = [{"question": data[i]['question'], "vector": vectors[i], "answer":  data[i]['answer']} for i in range(len(vectors)) ]

# res = client.insert(collection_name="faq_collection", data=data)
# res

In [80]:
# faq_queries = ["Quyết định 720/QĐ-CTN năm 2020"]

# faq_query_embeddings = bge_m3_ef.encode_queries(faq_queries)

# faq_query_embeddings

In [81]:
# res = client.search(
#     collection_name="faq_collection",  # target collection
#     data=[faq_query_embeddings['dense'][0]],  # query vectors
#     limit=5,  # number of returned entities
#     output_fields=["question", "answer"],  # specifies fields to be returned
# )
# context_items = res[0]

# print(context_items)

